In [1]:
import pandas as pd
import os
from tqdm.notebook import tqdm
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import texttable
import latextable
import json

tags = {
    "loss": "Loss_MB/train_phase/train_stream/Task000",
    "final_accuracy": "Accuracy_On_Trained_Experiences/eval_phase/test_stream/Task000",
}

def load_event_file(experiment_directory: str) -> EventAccumulator:
    """
    Loads the event file from a given experiment directory.
    """
    ea: EventAccumulator = EventAccumulator(experiment_directory)
    ea.Reload()
    return ea

def final_metrics(ea: EventAccumulator) -> dict:
    """
    Returns the final metrics from the event file.
    """
    result = {}
    for tag_name, tag in tags.items():
        if tag in ea.Tags()['scalars']:
            result[tag_name] = ea.Scalars(tag)[-1].value
    return result



In [3]:
experiment_logs = "experiment_logs"

records = []

# Loop over each directory in a directory
for experiment in tqdm(os.listdir(os.path.join(experiment_logs))):
    experiment_code = experiment.split("_")
    i, host, repo_hash, experiment_category, dataset, arch, strategy = experiment_code


    record = {
        "dataset": dataset,
        "architecture": arch,
        "strategy": strategy,
        "experiment_category": experiment_category,
        "id": i,
        "repo_hash": repo_hash
    }

    try:
        ea = load_event_file(os.path.join(experiment_logs, experiment))
        record.update(final_metrics(ea))
    except KeyError as e:
        print(f"Could not load required metric {e} in '{experiment}'")

    try:
        with open(os.path.join(experiment_logs, experiment, "config.json"), "r") as f:
            config = json.load(f)
            record.update(config)
    except FileNotFoundError as e:
        print(f"Could not load config file '{experiment}/config.json'")

    records.append(record)

df = pd.DataFrame.from_dict(records)
df

  0%|          | 0/167 [00:00<?, ?it/s]

Could not load config file '0006_ml-24_dd88ddf_N_splitFMNIST_VAE_taskInference/config.json'
Could not load config file '0084_ml-24_dd88ddf_N_splitEmbeddedCIFAR100_AE_taskInference/config.json'
Could not load config file '0101_ml-24_dd88ddf_N_splitEmbeddedCIFAR100_AE_taskOracle/config.json'
Could not load config file '0004_ml-24_dd88ddf_N_splitFMNIST_VAE_taskOracle/config.json'
Could not load config file '0005_ml-24_dd88ddf_N_splitFMNIST_AE_taskInference/config.json'
Could not load config file '0082_ml-24_dd88ddf_N_splitEmbeddedCIFAR100_AE_taskInference/config.json'
Could not load config file '0086_ml-24_dd88ddf_N_splitEmbeddedCIFAR100_AE_taskInference/config.json'
Could not load config file '0003_ml-24_dd88ddf_N_splitFMNIST_AE_taskOracle/config.json'
Could not load config file '0081_ml-24_dd88ddf_N_splitEmbeddedCIFAR100_AE_taskInference/config.json'
Could not load config file '0001_ml-24_dd88ddf_N_splitFMNIST_AE_cumulative/config.json'
Could not load config file '0002_ml-24_dd88ddf_N_s

,dataset,architecture,strategy,experiment_category,id,repo_hash,final_accuracy,batch_size,classifier_loss_weight,dataset_name,...,task_inference_strategy,total_task_epochs,use_classifier_loss,use_packnet,use_reconstruction_loss,use_vae_loss,rectangular_mlp_config,loss,vae_loss_weight,vanilla_cnn_config
0,splitCIFAR100,AE,taskOracle,EP,0003,b7711e62,0.566900,64.0,1.0,CIFAR100,...,task_oracle,100.0,True,True,True,False,NaN,NaN,NaN,NaN
1,splitEmbeddedCIFAR100,AE,taskInference,N,0076,dd88ddf,NaN,64.0,1.0,CIFAR100,...,task_reconstruction_loss,20.0,True,True,True,False,"{'depth': 4, 'width': 512}",NaN,NaN,NaN
2,splitCIFAR100,AE,taskOracle,EP,0136,dd88ddf,0.556700,64.0,1.0,CIFAR100,...,task_oracle,100.0,True,True,True,False,NaN,NaN,NaN,NaN
3,splitEmbeddedCORe50,AE,taskInference,PL,0008,7ee898bd,0.441119,64.0,1.0,CORe50_NC,...,task_reconstruction_loss,10.0,True,True,True,False,NaN,NaN,NaN,NaN
4,splitEmbeddedCIFAR100,AE,taskInference,N,0087,dd88ddf,0.762000,64.0,1.0,CIFAR100,...,task_reconstruction_loss,50.0,True,True,True,False,NaN,0.638947,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,splitCIFAR100,AE,finetuning,N,0025,dd88ddf,0.089900,64.0,1.0,CIFAR100,...,NaN,100.0,True,False,True,False,NaN,0.291340,NaN,{'base_channels': 128}
163,splitFMNIST,AE,taskInference,PL,0150,55dff8e1,0.994000,64.0,1.0,FMNIST,...,task_reconstruction_loss,20.0,True,True,True,False,NaN,NaN,NaN,NaN
164,splitCORe50,AE,finetuning,N,0026,dd88ddf,0.074535,64.0,1.0,CORe50_NC,...,NaN,4.0,True,False,True,False,NaN,0.307392,NaN,{'base_channels': 128}
165,splitEmbeddedCIFAR100,VAE,finetuning,N,0064,dd88ddf,0.077600,64.0,1.0,CIFAR100,...,NaN,50.0,True,False,True,True,NaN,0.190293,0.001,NaN


In [3]:
df.dropna().sort_values(by='id').groupby(['dataset', 'architecture', 'strategy']).max().pivot_table(index=['architecture', 'strategy'], columns=['dataset'], values='final_accuracy').to_csv('accuracy.csv')

In [4]:
df.to_csv("results/results.csv")

In [32]:
# 
# Regular Experiment
# 
for strategy in ["cumulative", "taskOracle", "finetuning", "taskInference"]:
    for arch in ["AE", "VAE"]:
        for dataset in [
            "splitFMNIST",
            "splitCIFAR10",
            "splitCIFAR100",
            "splitCORe50",
            "splitEmbeddedCIFAR100",
            "splitEmbeddedCORe50"]:
            try:
                result = df[df['dataset'] == dataset]
                result = result[result['prune_strategy'] == "EXP"]
                result = result[result['strategy'] == strategy]
                result = result[result['architecture'] == arch]
                accuracy = result.sort_values(by='id').iloc[-1]["final_accuracy"]
                # accuracy = result.sort_values("id")[-1]["accuracy"]
                # display(result.sort_values(by='id'))
                print(f"{accuracy:0.4f}", end="\t")
            except:
                continue
        print()

0.9151	0.8721	0.6167	0.3560	0.4210	0.7720	
0.8989	0.8725	0.6106	0.3604	0.3974	0.7593	
0.9938	0.9375	0.5475	0.4322	0.5513	0.6042	
0.9936	0.9382	0.5910	0.4032	0.4735	0.5191	
0.1995	0.1939	0.0899	0.0745	0.0841	0.0881	
0.1995	0.1945	0.0891	0.0751	0.0849	0.0578	
0.9597	0.7123	0.2351	0.0553	0.4392	0.4191	
0.9927	0.7295	0.3091	0.0531	0.3626	0.3756	


In [39]:
# 
# Equal Prune Experiment
# 
def table_equal_prune(experiment_results: pd.DataFrame):
    df = experiment_results.copy()

    equal_prune_table = texttable.Texttable()
    columns = ["splitFMNIST", "splitCIFAR10", "splitCIFAR100", "splitCORe50", "splitEmbeddedCIFAR100", "splitEmbeddedCORe50"]
    
    equal_prune_table.header(["", ""] + columns)

    for strategy in ["taskOracle", "taskInference"]:
        for arch in ["AE", "VAE"]:
            row = [strategy, arch]
            for dataset in columns:
                try:
                    result = df.loc[
                        (df['dataset'] == dataset) &
                        (df['prune_strategy'] == "EP") &
                        (df['strategy'] == strategy) &
                        (df['architecture'] == arch)
                    ]
                    accuracy = result.sort_values(by='id').iloc[-1]["final_accuracy"]
                    row.append(f"{accuracy*100:0.1f}\%")
                except:
                    row.append(f"NA")
            equal_prune_table.add_row(row)

    print(equal_prune_table.draw())

    print(latextable.draw_latex(
        equal_prune_table,
        caption="Equal Prune Experiment",
        label="tab:equal_prune_experiment",
        use_booktabs=True
    ))

table_equal_prune(df[df["prune_strategy"]=="EP"])

+----------+-----+----------+----------+---------+---------+---------+---------+
|          |     | splitFMN | splitCIF | splitCI | splitCO | splitEm | splitEm |
|          |     |   IST    |   AR10   | FAR100  |  Re50   | beddedC | beddedC |
|          |     |          |          |         |         | IFAR100 |  ORe50  |
+==========+=====+==========+==========+=========+=========+=========+=========+
| taskOrac | AE  | 99.3\%   | 93.4\%   | 55.7\%  | 50.7\%  | NA      | NA      |
| le       |     |          |          |         |         |         |         |
+----------+-----+----------+----------+---------+---------+---------+---------+
| taskOrac | VAE | 99.3\%   | 91.7\%   | 53.0\%  | 54.1\%  | NA      | NA      |
| le       |     |          |          |         |         |         |         |
+----------+-----+----------+----------+---------+---------+---------+---------+
| taskInfe | AE  | 99.3\%   | 43.0\%   | 25.7\%  | 8.2\%   | NA      | NA      |
| rence    |     |          

In [7]:
# 
# Equal Prune Experiment
# 
def table_prune_levels(experiment_results: pd.DataFrame):
    df = experiment_results.copy()

    equal_prune_table = texttable.Texttable()
    columns = ["splitFMNIST", "splitCIFAR10", "splitCIFAR100", "splitCORe50", "splitEmbeddedCIFAR100", "splitEmbeddedCORe50"]
    
    equal_prune_table.header(["Prune Level"] + columns)

    for prune_level in [0.2, 0.4, 0.5, 0.6, 0.8]:
            row = [f"{prune_level*100:0.0f}\%"]
            for dataset in columns:
                try:
                    result = df.loc[
                        (df['dataset'] == dataset) &
                        (df['prune_proportion'] == prune_level) &
                        (df['architecture'] == "AE")
                    ]
                    accuracy = result.sort_values(by='id').iloc[-1]["final_accuracy"]
                    row.append(f"{accuracy*100:0.1f}\%")
                except:
                    row.append(f"NA")
            equal_prune_table.add_row(row)

    print(equal_prune_table.draw())

    print(latextable.draw_latex(
        equal_prune_table,
        caption="Multiple Prune Levels with Task Inference AutoEncoder",
        label="tab:prune_levels",
        use_booktabs=True
    ))

table_prune_levels(df[
    (df["experiment_category"]=="PL") & 
    ((df["repo_hash"] == "55dff8e1") | (df["repo_hash"] == "7ee898bd"))
])


+-----------+-----------+----------+----------+----------+----------+----------+
|   Prune   | splitFMNI | splitCIF | splitCIF | splitCOR | splitEmb | splitEmb |
|   Level   |    ST     |   AR10   |  AR100   |   e50    | eddedCIF | eddedCOR |
|           |           |          |          |          |  AR100   |   e50    |
+===========+===========+==========+==========+==========+==========+==========+
| 20\%      | 73.6\%    | 36.7\%   | 17.1\%   | 6.7\%    | 17.4\%   | 21.7\%   |
+-----------+-----------+----------+----------+----------+----------+----------+
| 40\%      | 99.4\%    | 43.5\%   | 21.5\%   | 5.9\%    | 33.5\%   | 35.1\%   |
+-----------+-----------+----------+----------+----------+----------+----------+
| 50\%      | 99.3\%    | 56.3\%   | 28.7\%   | 6.4\%    | 40.6\%   | 44.1\%   |
+-----------+-----------+----------+----------+----------+----------+----------+
| 60\%      | 99.4\%    | 78.6\%   | 17.3\%   | 9.0\%    | 51.1\%   | 54.8\%   |
+-----------+-----------+---